<a href="https://colab.research.google.com/github/nak650228/ITEC/blob/20211010/VideoRestore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#◢ Video Restoration Prework

プリワークでは、Youtubeまたはファイル指定により動画ファイルをダウンロードした後、DeepLearningによる補正を行う前処理を行います。
その他にも以下の修正を施します。
  
・フレームのスタビライズ（手振れ補正）  
・アップスケール/ダウンスケール（1080pに）  
・ヒストグラムの調整  
・シーンの分割（トランジションを判断して、複数の動画ファイルに分割します。  
  
出力結果は1080pの動画として、ユーザのGoogle Drive上のフォルダに作られます。

#◢ 初期設定

In [6]:
#@title 割り当てられたGPUの確認
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla P100-PCIE-16GB, 460.32.03, 16280 MiB


In [7]:
#@title ライブラリ等のインストール
%cd /content
!pip install youtube_dl
!pip install ffmpeg
!pip install ffmpeg-python
#!pip install torchvision==0.5
!pip install torchvision
#!pip install torch==1.4
!pip install torch==1.9.0
#!pip install scipy==1.2.0
!pip install scipy
#!pip install imgaug==0.2.5
!pip install imgaug
#!pip install tensorflow==1.15.5
!pip install tensorflow

#シーン分割
!pip install scenedetect[opencv,progress_bar]

!pip install subprocess

import tensorflow as tf
import youtube_dl
import ffmpeg
import numpy as np
import imageio
import cv2
import torch
import glob
import shutil
import moviepy.editor as mpy
import os
from IPython.display import clear_output
torch.backends.cudnn.benchmark=True

clear_output()


In [8]:
#@title 超解像度用ライブラリのインストール（realESRGAN,BSRGAN,SwinIR)

SRGAN_DIR = "/content/Real-ESRGAN"
if (not os.path.isdir(SRGAN_DIR)):
  # Clone realESRGAN
  !git clone https://github.com/xinntao/Real-ESRGAN.git
  %cd Real-ESRGAN
  # Set up the environment
  !pip install basicsr
  !pip install facexlib
  !pip install gfpgan
  !pip install -r requirements.txt
  !python setup.py develop

  # Clone BSRGAN
  !git clone https://github.com/cszn/BSRGAN.git

  # Clone SwinIR
  !git clone https://github.com/JingyunLiang/SwinIR.git
  !pip install timm

  # Download the pre-trained models
  !wget https://github.com/cszn/KAIR/releases/download/v1.0/BSRGAN.pth -P BSRGAN/model_zoo
  !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
  !wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth -P experiments/pretrained_models

clear_output()

In [9]:
#@title 手振れ補正用ライブラリ（SIUN)のインストール

%cd /content

SIUN_DIR = "/content/SIUN"

if (not os.path.isdir(SIUN_DIR)):
  !git clone https://github.com/minyuanye/SIUN.git

  %cd /content/SIUN/code
  !pip install h5py==2.7.1
  !pip install tensorflow-gpu=1.15.0
  !pip install Keras==2.2.4
  !pip install scikit-image==0.14.3

clear_output()

In [10]:
#@title 手振れ補正用ライブラリの（vidstab)のインストール

!pip install vidstab[cv2]

clear_output()

In [11]:
#@title Microsoft Bringing-Old-Photos-Back-to-Lifeのリポジトリーをクローンする
%cd /content
!git clone https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life.git photo_restoration

clear_output()

In [12]:
#@title Microsoft Bringing Old-Photos-Back-to-Lifeの学習済みモデルをダウンロード
# pull the syncBN repo
%cd photo_restoration/Face_Enhancement/models/networks
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../../

%cd Global/detection_models
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../

# download the landmark detection model
%cd Face_Detection/
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2
%cd ../

# download the pretrained model
%cd Face_Enhancement/
!wget https://facevc.blob.core.windows.net/zhanbo/old_photo/pretrain/Face_Enhancement/checkpoints.zip
!unzip -o checkpoints.zip
%cd ../

%cd Global/
!wget https://facevc.blob.core.windows.net/zhanbo/old_photo/pretrain/Global/checkpoints.zip
!unzip -o checkpoints.zip
%cd ../

! pip install -r requirements.txt

clear_output()

In [13]:
#@title DeOldifyの初期設定
%cd /content

!git clone https://github.com/jantic/DeOldify.git DeOldify
%cd DeOldify

#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

from os import path

!pip install -r colab_requirements.txt

import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*?Your .*? set is empty.*?")

!mkdir 'models'
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth

colorizer = get_video_colorizer()

clear_output()

In [14]:
#@title ##**GFPGANをGithubからクローン**

# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
clear_output()

In [15]:
#@title ##**Clone the repository of DeepRemaster** { display-mode: "form" }
%cd /content
!git clone https://github.com/satoshiiizuka/siggraphasia2019_remastering.git DeepRemaster
!cp -r /content/video.mp4 /content/DeepRemaster/
%cd /content/DeepRemaster

!wget --continue -O model/remasternet.pth.tar -- http://iizuka.cs.tsukuba.ac.jp/data/remasternet.pth.tar
clear_output()

#◢ 関数定義

In [16]:
#@title ##**ビデオを静止画フレームに分解する** { display-mode: "form" }

frame_folder  = "/content/Real-ESRGAN/BSRGAN/testsets/RealSRSet"

if os.path.isdir(frame_folder):
    shutil.rmtree(frame_folder)

os.mkdir(frame_folder)

os.chdir(frame_folder)

!ffmpeg -i /content/video.mp4 %09d.png

clear_output()

#◢ 画像のダウンロード

In [ ]:
#@title **Googleドライブの追加**
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print('Google Drive connected.')

Mounted at /content/drive
Google Drive connected.


In [36]:
#@title ##**ビデオのダウンロード** { display-mode: "form" }
#@markdown *ビデオへのリンク（YouTubeやTwitterなど）を入力するか、source_urlフィールドを空白にしてください（空白にした場合、コンピューターからビデオをアップロードするよう求められます）。*
#@markdown *プロジェクト名は任意でOKです。一応最後にGoogle Driveにプロジェクト名で指定したディレクトリが作られて、途中経過も含めて全ての動画ファイルがコピーされます*

import youtube_dl
import cv2
from IPython.display import clear_output

%cd /content

projectname = 'Downtown18' #@param {type:"string"}

source_url = 'https://www.youtube.com/watch?v=jenWdylTtzs' #@param {type:"string"}

%cd /content
! rm -f /content/*.mp4

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "downloaded_video." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    file_name = "downloaded_video." + fn.split(".")[-1]
    !mv -f $fn $file_name

!cp -r /content/downloaded_video.mp4 /content/video.mp4

clear_output()

fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
width_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_WIDTH))
height_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_HEIGHT))
print ("FPS of VIDEO: ",fps_of_video)
print ("Frames of VIDEO: ",frames_of_video)
print ("Width of VIDEO: ",width_of_video)
print ("Height of VIDEO: ",height_of_video)



FPS of VIDEO:  29
Frames of VIDEO:  2080
Width of VIDEO:  720
Height of VIDEO:  480


In [ ]:
#@title ##**ダウンロードした動画を表示** { display-mode: "form" }
#@markdown *ビデオは横640ドットに拡大/縮小されて表示されます*
import moviepy.editor as mpy

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/downloaded_video.mp4", autoplay=1, maxduration=6000,width=640))
else:
  files.download('/content/downloaded_video.mp4')

In [37]:
#@markdown *1分以上の動画をダウンロードすることはお勧めできません。また、タイトルに「スペース」や「ドット」が含まれている動画はアップロードしないでください。*

#@markdown *実行中にエラーが発生した場合は、このブロックを再度実行します。*

#@title ##**動画の調整** { display-mode: "form" }
#@markdown *動画の長さを変更することができます。*

#@markdown **動画の長さを変更する場合は、その開始時間と終了時間を指定して下さい。**
target_start = '00:00:00' #@param {type:"string"}
target_end = '00:01:00' #@param {type:"string"}


if os.path.isfile("/content/cropped_video.mp4"):
    os.remove("/content/cropped_video.mp4")

!ffmpeg -i /content/downloaded_video.mp4  -ss $target_start -to $target_end /content/cropped_video.mp4

if os.path.isfile("/content/video.mp4"):
    os.remove("/content/video.mp4")

!cp /content/cropped_video.mp4 /content/video.mp4

#@markdown **モノクロ動画にAIで色を付ける場合は有効にしてください。**
is_deoldify = True #@param {type:"boolean"}


#@markdown **Deoldifyのレンダリングファクターを指定します。(お勧めは10～25）**
render_factor = 13  #@param {type: "slider", min: 5, max: 44}


#@markdown **ヒストグラムの平均化を行う場合、動画の種類に合わせてモノクロかカラーを選択してください。**
#@markdown **ヒストグラムの平均化を行わない場合はNoneを選択してください。**
#@markdown **ただしこれまでの経験では有効にしてもあまりきれいにはなりませんでした**
HistgramType = 'None' #@param ["None", "Monochrome", "Color"] {allow-input: true}


#@markdown **DeepRemasterを有効にすると、低画質の動画からノイズなどを除去できます。**
is_DeepRemaster = True #@param {type:"boolean"}

#@markdown **GANを使って、動画の顔部分を書き換えます。大体はがっかりしますが、時々きれいに修正されることがあります**

#@markdown **Microsoft Bringing-Old-Photos-Back-to-LifeまたはGFPGANを選択して、いずれかの手法で人物の精彩化を試みます。**
which_FaceGAN = 'Microsoft' #@param ["None", "GFPGAN", "Microsoft"] {allow-input: true}

clear_output()

In [ ]:
#@title ##**サイズ調整後の動画を表示（修復対象）** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/video.mp4')

In [39]:
!scenedetect --input /content/video.mp4 detect-content list-scenes save-images

[PySceneDetect] PySceneDetect v0.5.6.1
[PySceneDetect] Loaded 1 video, framerate: 29.970 FPS, resolution: 720 x 480
[PySceneDetect] Downscale factor set to 3, effective resolution: 240 x 160
[PySceneDetect] Scene list CSV file name format:
  $VIDEO_NAME-Scenes.csv
[PySceneDetect] Image output format set: JPEG [Quality: 95]
[PySceneDetect] Detecting scenes...
100% 1799/1799 [00:04<00:00, 445.75frames/s]
[PySceneDetect] Processed 1799 frames in 4.0 seconds (average 445.09 FPS).
[PySceneDetect] Detected 11 scenes, average shot length 5.5 seconds.
[PySceneDetect] Writing scene list to CSV file:
  video-Scenes.csv
[PySceneDetect] Scene List:
-----------------------------------------------------------------------
 | Scene # | Start Frame |  Start Time  |  End Frame  |   End Time   |
-----------------------------------------------------------------------
 |      1  |           0 | 00:00:00.000 |         108 | 00:00:03.604 |
 |      2  |         108 | 00:00:03.604 |         191 | 00:00:06.373 

# #◢ 小さな動画を1080Pに拡大
ここではSwinIRという超高解像度の手法を使っていますが。これは基本的に静止画用です。今後TecoGANもしくはiseebetterに置き換える予定です。

In [ ]:
#@title ##**動画のスケーリング** { display-mode: "form" }
##@markdown *動画の解像度を変更することができます。*

##@markdown **動画の解像度を変更する場合は、新しい解像度を指定してください（例 640 x 480）。この値を指定しない場合は、元の解像度がそのまま使われます。**

if height_of_video  < 1080:
  scale_factor = 4

#width =  720#@param {type:"number"}
#height =  480#@param {type:"number"}



In [ ]:
#@title ##**ビデオを静止画フレームに分解する** { display-mode: "form" }

frame_folder  = "/content/Real-ESRGAN/BSRGAN/testsets/RealSRSet"

if os.path.isdir(frame_folder):
    shutil.rmtree(frame_folder)

os.mkdir(frame_folder)

os.chdir(frame_folder)

!ffmpeg -i /content/video.mp4 %09d.png

clear_output()

In [ ]:
#@title ##**SwinIRによる動画の拡大** { display-mode: "form" }

os.chdir("/content/Real-ESRGAN")
upscale_command="python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth --folder_lq BSRGAN/testsets/RealSRSet --scale " + str(scale_factor)
subprocess.run(upscale_command,shell=True)

#!python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth --folder_lq BSRGAN/testsets/RealSRSet --scale 
shutil.move('results/swinir_real_sr_x4', 'results/SwinIR')

FileNotFoundError: ignored

In [ ]:
#@title ##**修復した静止画を動画に復元** { display-mode: "form" }
#!ffmpeg -vsync 0 -hwaccel cuvid -c:v mjpeg_cuvid -framerate 30 -i /content/TecoGAN/results/My_video/*.png -c:v h264_nvenc quaid2.mp4

upscaled_file="/content/upscaled_video.mp4"
result_folder="/content/Real-ESRGAN/results/SwinIR/"

if os.path.isfile(upscaled_file) :
    os.remove(upscaled_file)

os.chdir(result_folder)

subprocess.run('/usr/bin/ffmpeg -f image2 -framerate ' + str(fps_of_video) + ' -i /content/Real-ESRGAN/results/SwinIR/%09d_SwinIR.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/upscaled_video.mp4' , shell=True )
#!ffmpeg -f image2 -framerate fps_of_video -i %09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/histgram_video.mp4
#!rm -f /content/histgram_video.mp4
!cp /content/upscaled_video.mp4 /content/video.mp4

#clear_output()

In [ ]:
#@title ##**拡大した動画を1080pに変換** { display-mode: "form" }

width = int(width_of_video *  1080 / height_of_video)
height = 1080

rescale = ""
if width != '' and height != '':
  rescale = f"-s {width}x{height}"
  rescale = f"-vf scale={width}:{height}"

!ffmpeg -i /content/upscaled_video.mp4 $rescale /content/resized_video.mp4

if os.path.isfile("/content/video.mp4"):
    os.remove("/content/video.mp4")

!cp /content/resized_video.mp4 /content/video.mp4

clear_output()

# #◢ モノクロ動画に色を付ける

In [ ]:
#@title Deoldifyによるモノクロ動画のカラー化
if is_deoldify == True:

  %cd /content/DeOldify

  if os.path.isfile("/content/DeOldify/video"):
    shutil.rmtree("/content/DeOldify/video")

  !mkdir -p '/content/DeOldify/video/source'

  !cp -r /content/video.mp4 /content/DeOldify/video/source/video.mp4
  video_path = colorizer.colorize_from_file_name('/content/DeOldify/video/source/video.mp4', render_factor)
  !cp -r /content/DeOldify/video/result/video.mp4 /content/colorized_video.mp4
  !cp -r /content/colorized_video.mp4 /content/video.mp4
  if os.path.isfile("/content/DeOldify/video/result/video.mp4"):
    os.remove("/content/DeOldify/video/result/video.mp4")


/content/DeOldify


Video created here: video/result/video.mp4


In [ ]:
#@title ##**カラー化後の表示** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/colorized_video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/colorlized_video.mp4')

# #◢ ノイズ除去(Deep Remaster)

In [ ]:
#@title ##**Remove frame noise** { display-mode: "form" }
%cd /content/DeepRemaster
command = "python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim "+str(height)

#subprocess.run(command,shell=True)
!python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim 1080
#!python remaster.py --input /content/video.mp4 --disable_colorization --gpu
#clear_output()

/content/DeepRemaster


CompletedProcess(args='python remaster.py --input /content/video.mp4 --disable_colorization --gpu --mindim 1080', returncode=0)

In [ ]:
#@title ##**ノイズ除去結果の表示** { display-mode: "form" }
!rm -rf /content/video.mp4
!cp -r video_out.mp4 /content/video.mp4
!cp -r video_out.mp4 /content/denoise_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/denoise_video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/denoise_video.mp4')

#◢ 画像の修復(ヒストグラムの修正）

In [ ]:
#@title ##**ビデオを静止画フレームに分解する** { display-mode: "form" }

if HistgramType != "None":
  upload_folder = "/content/datas"
  frame_folder  = "/content/datas/frames"

  if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)

  os.mkdir(upload_folder)
  os.mkdir(frame_folder)

  os.chdir(frame_folder)

  !ffmpeg -i /content/video.mp4 %09d.png

  clear_output()

In [ ]:
#@title ##**白黒画像のヒストグラムを平坦化する** { display-mode: "form" }

if is_Histgram == "Monochrome":
  files = os.listdir(frame_folder)

  for file in files:
    img = cv2.imread(file,0)

  # create a CLAHE object (Arguments are optional).
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(img)

    cv2.imwrite(file,cl1)

In [ ]:
#@title ##**カラー画像のヒストグラムを平坦化する** { display-mode: "form" }
import cv2

# ヒストグラム平坦化でコントラスト強調                                                                                                      
def eqh(img):
    # 各色で平坦化                                                                                                                          
    b1,g1,r1 = cv2.split(img)
    b2 = cv2.equalizeHist(b1)
    g2 = cv2.equalizeHist(g1)
    r2 = cv2.equalizeHist(r1)
    eqh_rgb = cv2.merge((b2,g2,r2))

    # HSVのvだけ編集                                                                                                                        
    h1,s1,v1 = cv2.split(cv2.cvtColor(img,cv2.COLOR_BGR2HSV))  # 色空間をBGRからHSVに変換                                                   
    v2 = cv2.equalizeHist(v1)
    eqh_hsv = cv2.cvtColor(cv2.merge((h1,s1,v2)), cv2.COLOR_HSV2BGR)

    return eqh_rgb, eqh_hsv

# 部分的にヒストグラム平坦化でコントラスト強調                                                                                              
def clc(img,cl,gsize):
    b1,g1,r1 = cv2.split(img)
    clahe = cv2.createCLAHE(clipLimit=cl, tileGridSize=(gsize,gsize))
    b2 = clahe.apply(b1)
    g2 = clahe.apply(g1)
    r2 = clahe.apply(r1)
    return cv2.merge((b2,g2,r2))

# 値を0-255にclipして、typeをuint8にする                                                                                                    
def ct(img):
    return np.clip(img,0,255).astype(np.uint8)


if HistgramType == "Color":
  files = os.listdir("/content/datas/frames")

  for file in files:
    img = cv2.imread(file)
    # コントラスト強調画像を作成 
    eqh_rgb,eqh_hsv = eqh(img)
    clc_img = clc(img,2.,4)
    # 全部混ぜる                                                                                                                                
    ave_img = ct((np.float32(img) \
            + np.float32(eqh_rgb) \
            + np.float32(eqh_hsv) \
            + np.float32(clc_img))/4.)


    cv2.imwrite(file,ave_img)



In [ ]:
#@title ##**修復した静止画を動画に復元** { display-mode: "form" }
#!ffmpeg -vsync 0 -hwaccel cuvid -c:v mjpeg_cuvid -framerate 30 -i /content/TecoGAN/results/My_video/*.png -c:v h264_nvenc quaid2.mp4

if HistgramType != "None":
  import subprocess
  histgram_file="/content/histgram_video.mp4"

  if os.path.isfile(histgram_file) :
    os.remove(histgram_file)

  os.chdir(frame_folder)

  subprocess.run('/usr/bin/ffmpeg -f image2 -framerate ' + str(fps_of_video) + ' -i /content/datas/frames/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/histgram_video.mp4' , shell=True )
#!ffmpeg -f image2 -framerate fps_of_video -i %09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/histgram_video.mp4
#!rm -f /content/histgram_video.mp4
  !cp /content/histgram_video.mp4 /content/video.mp4

#clear_output()

In [ ]:
#@title ##**ヒストグラム調整後の動画を表示** { display-mode: "form" }

what_next = 'play' #@param ["play", "download"]
if what_next == "play":
#  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
  display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600))
else:
  files.download('/content/video.mp4')

#◢ Microsoft Bringing-Old-Photos-Back-to-Lifeによる画像修正

In [ ]:
#@title 動画ファイルを画像ファイルに分解
# ffmpeg extract - Generating individual frame PNGs from the source file.

import shutil
if which_FaceGAN == "Microsoft":

  %cd /content/photo_restoration

  FRAME_INPUT_DIR = "/content/photo_restoration/input_frames"
  FRAME_OUTPUT_DIR = "/content/photo_restoration/output_frames"
  INPUT_FILEPATH = "/content/video.mp4"

  if os.path.isfile(FRAME_INPUT_DIR):
    shutil.retree(FRAME_INPUT_DIR)

  %shell mkdir -p '{FRAME_INPUT_DIR}'

  %shell ffmpeg -i '{INPUT_FILEPATH}' '{FRAME_INPUT_DIR}/%05d.png'

  png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
  from IPython.display import clear_output

  pngs_generated_count = int(png_generated_count_command_result.output.strip())


  #print(f"Input FPS: {fps}")
  print(f"{pngs_generated_count} frame PNGs generated.")

  # Checking if PNG do have alpha
  import subprocess as sp
  %cd {FRAME_INPUT_DIR}
  channels = sp.getoutput('identify -format %[channels] 00001.png')
  print (f"{channels} detected")

  # Removing alpha if detected
  if "a" in channels:
    print("Alpha detected and will be removed.")
    print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

  clear_output()

In [ ]:
#@title 精彩化の実行

if which_FaceGAN == "Microsoft":
  %cd /content/photo_restoration
  input_folder = FRAME_INPUT_DIR
  output_folder = FRAME_OUTPUT_DIR

  !rm -rf /content/photo_restoration/output_frames/*

  print (input_folder)
  print (output_folder)

  import os
  basepath = os.getcwd()
  #input_path = os.path.join(basepath, input_folder)
  #output_path = os.path.join(basepath, output_folder)
  #os.mkdir(output_path)
  #!rm -rf output_folder
  #os.mkdir(output_folder)

  !python run.py --input_folder /content/photo_restoration/input_frames --output_folder /content/photo_restoration/output_frames --GPU 0

  clear_output()

In [ ]:
#@title ビデオファイルの作成
#create video

if which_FaceGAN == "Microsoft":
  %cd /content/photo_restoration/output_frames/final_output
  !ffmpeg  -pattern_type glob -i '*.png' -c:v h264_nvenc -pix_fmt yuv420p /content/beautified_video.mp4
  !cp /content/output.mp4 /content/drive/MyDrive/Movie

  clear_output()

In [ ]:
#@title ##**Get result** { display-mode: "form" }
if which_FaceGAN == "Microsoft":
  !rm -rf /content/video.mp4
  !cp -r /content/beautified_video.mp4 /content/video.mp4
  what_next = 'play' #@param ["play", "download"]
  if what_next == "play":
    display(mpy.ipython_display("/content/beautified_video.mp4", autoplay=1,  maxduration=600))
  else:
    files.download('/content/beautified_video.mp4')

#◢ GFPGANによる顔画像の修復

In [ ]:
#@title ##**ビデオを静止画フレームに分解する** { display-mode: "form" }

if which_FaceGAN == "GFPGAN":

  upload_folder = "/content/GFPGAN/inputs/upload"
  if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)

  os.mkdir(upload_folder)

  %cd /content/GFPGAN/inputs/upload

  !ffmpeg -i /content/video.mp4 %09d.png

  clear_output()

In [ ]:
#@title ##**GFPGANによる修復** { display-mode: "form" }
# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions
if which_FaceGAN == "GFPGAN":
  %cd /content/GFPGAN
  !rm -rf results
  !python inference_gfpgan.py --upscale 2 --test_path inputs/upload --save_root results --model_path experiments/pretrained_models/GFPGANCleanv1-NoCE-C2.pth --bg_upsampler realesrgan
  clear_output()

  !ls results/cmp

In [ ]:
#@title ##**修復した静止画を動画に復元** { display-mode: "form" }
#!ffmpeg -vsync 0 -hwaccel cuvid -c:v mjpeg_cuvid -framerate 30 -i /content/TecoGAN/results/My_video/*.png -c:v h264_nvenc quaid2.mp4
if which_FaceGAN == "GFPGAN":
  if os.path.isfile("/content/restored_video.mp4") :
    !rm -f /content/restored_video.mp4

  !ffmpeg -f image2 -framerate 30 -i /content/GFPGAN/results/restored_imgs/%09d.png -c:v h264_nvenc -preset slow -qp 18 -pix_fmt yuv420p /content/restored_video.mp4
  !rm -f /content/video.mp4
  !cp /content/restored_video.mp4 /content/video.mp4

  clear_output()

In [ ]:
#@title ##**修復した動画を表示** { display-mode: "form" }
#@markdown *what_nextにplayを指定すると、GFPGANで修復した結果を表示します。解像度が大きな動画は表示が失敗することがあります。その場合はwhat_nextにdownloadを指定して、PCなどにダウンロードして確認して下さい*

if which_FaceGAN == "GFPGAN":
  what_next = 'play' #@param ["play", "download"]
  if what_next == "play":
    #  display(mpy.ipython_display("/content/video.mp4", height=400, autoplay=1, maxduration=600))
    display(mpy.ipython_display("/content/video.mp4", autoplay=1, maxduration=600,width=640))
  else:
    files.download('/content/video.mp4')

#◢ 最終処理

In [ ]:
#@title ファイルのバックアップ

ProjectDir="/content/drive/MyDrive/Movie/"+str(projectname)
print("ProjectDir")

if os.path.isfile(ProjectDir):
    shutil.rmtree(ProjectDir)

os.mkdir(ProjectDir)
os.chdir(ProjectDir)
!mv /content/*.mp4 .

ProjectDir
